In [4]:
# Load the data

In [13]:
import pandas as pd
import os

# Check if we are inside the 'utils' folder
if os.path.basename(os.getcwd()) == 'utils':
    path = '../data/sales_data.csv'
else:
    path = 'data/sales_data.csv'

df = pd.read_csv(path, sep='|')
df.head()

,TransactionID,Date,ProductID,ProductName,Quantity,UnitPrice,CustomerID,Region
0,T018,2024-12-29,P107,USB Cable,8,173,C009,South
1,T063,2024-12-07,P110,Laptop Charger,6,"1,916",C022,East
2,T075,2024-12-10,P106,Headphones,0,2826,C001,South
3,T023,2024-12-09,P109,Wireless Mouse,9,523,C022,North
4,T059,2024-12-29,P102,"Mouse,Wireless",4,1056,C010,South


In [14]:
df.shape

(80, 8)

In [15]:
total_records = len(df)


In [16]:
df = df.dropna(subset=["CustomerID", "Region"])


In [17]:
df = df[df["Quantity"] > 0]


In [18]:
df["UnitPrice"] = df["UnitPrice"].astype(str).str.replace(",", "")


In [19]:
df["UnitPrice"] = pd.to_numeric(df["UnitPrice"], errors="coerce")


In [20]:
df = df[df["UnitPrice"] > 0]


In [21]:
df = df[df["TransactionID"].str.startswith("T")]


In [22]:
df["ProductName"] = df["ProductName"].str.replace(",", "", regex=False)


In [23]:
df["Quantity"] = df["Quantity"].astype(str).str.replace(",", "").astype(int)
df["UnitPrice"] = df["UnitPrice"].astype(str).str.replace(",", "").astype(float)


In [24]:
valid_records = len(df)
invalid_records = total_records - valid_records


In [25]:
print(f"Total records parsed: {total_records}")
print(f"Invalid records removed: {invalid_records}")
print(f"Valid records after cleaning: {valid_records}")


Total records parsed: 80
Invalid records removed: 10
Valid records after cleaning: 70


In [26]:
#Task 1.1 — Read Sales Data with Encoding Handling

In [27]:
def read_sales_data(filename):
    """
    Reads sales data from file handling encoding issues.

    Returns:
        list of raw transaction lines (strings)

    Expected Output Format:
        ['T001|2024-12-01|P101|Laptop|2|45000|C001|North', ...]
    """
    encodings = ["utf-8", "latin-1", "cp1252"]

    for encoding in encodings:
        try:
            with open(filename, "r", encoding=encoding) as file:
                lines = file.readlines()

            # Skip header and remove empty lines
            data_lines = [
                line.strip()
                for line in lines[1:]
                if line.strip()
            ]

            return data_lines

        except UnicodeDecodeError:
            continue

        except FileNotFoundError:
            print(f"Error: File '{filename}' not found.")
            return []

    print("Error: Unable to read file with supported encodings.")
    return []


In [30]:
#import os

#def read_sales_data(file_path):
    # Move to project root (parent of utils)
   # project_root = os.path.dirname(os.getcwd())
   # full_path = os.path.join(project_root, file_path)

   # if not os.path.exists(full_path):
       # print(f"Error: File '{full_path}' not found.")
       # return []
#
   # with open(full_path, 'r', encoding='utf-8') as f:
       # return f.readlines()


In [28]:
import os

def read_sales_data(file_path):
    # Always resolve from project root
    current_dir = os.getcwd()

    # If running from utils/, go one level up
    if current_dir.endswith("utils"):
        project_root = os.path.dirname(current_dir)
    else:
        project_root = current_dir

    full_path = os.path.join(project_root, file_path)

    if not os.path.exists(full_path):
        print(f"Error: File '{full_path}' not found.")
        return []

    with open(full_path, "r", encoding="utf-8") as f:
        return f.readlines()


In [29]:
sales_data = read_sales_data("data/sales_data.csv")

In [30]:
print("Total transactions read:", len(sales_data))
print("First 3 records:")
sales_data[:3]


Total transactions read: 81
First 3 records:


['TransactionID|Date|ProductID|ProductName|Quantity|UnitPrice|CustomerID|Region\n',
 'T018|2024-12-29|P107|USB Cable|8|173|C009|South\n',
 'T063|2024-12-07|P110|Laptop Charger|6|1,916|C022|East\n']

In [31]:
#Task 1.2 — Parse and Clean Data

In [32]:
def parse_transactions(raw_lines):
    """
    Parses raw transaction lines into a clean list of dictionaries.
    """
    parsed_data = []

    for line in raw_lines:
        # Split by pipe delimiter
        parts = line.split("|")

        # Skip rows with incorrect number of fields
        if len(parts) != 8:
            continue

        transaction_id, date, product_id, product_name, quantity, unit_price, customer_id, region = parts

        # Handle commas in ProductName
        product_name = product_name.replace(",", "").strip()

        # Handle commas in numeric fields
        quantity = quantity.replace(",", "").strip()
        unit_price = unit_price.replace(",", "").strip()

        try:
            # Convert data types
            quantity = int(quantity)
            unit_price = float(unit_price)
        except ValueError:
            # Skip rows with invalid numeric values
            continue

        record = {
            "TransactionID": transaction_id.strip(),
            "Date": date.strip(),
            "ProductID": product_id.strip(),
            "ProductName": product_name,
            "Quantity": quantity,
            "UnitPrice": unit_price,
            "CustomerID": customer_id.strip(),
            "Region": region.strip()
        }

        parsed_data.append(record)

    return parsed_data


In [33]:
parsed_transactions = parse_transactions(sales_data)


In [34]:
print("Parsed transactions:", len(parsed_transactions))
parsed_transactions[:2]


Parsed transactions: 80


[{'TransactionID': 'T018',
  'Date': '2024-12-29',
  'ProductID': 'P107',
  'ProductName': 'USB Cable',
  'Quantity': 8,
  'UnitPrice': 173.0,
  'CustomerID': 'C009',
  'Region': 'South'},
 {'TransactionID': 'T063',
  'Date': '2024-12-07',
  'ProductID': 'P110',
  'ProductName': 'Laptop Charger',
  'Quantity': 6,
  'UnitPrice': 1916.0,
  'CustomerID': 'C022',
  'Region': 'East'}]

In [35]:
import os

def read_sales_data(file_path):
    if not os.path.isabs(file_path):
        file_path = os.path.join(os.getcwd(), file_path)

    if not os.path.exists(file_path):
        print(f"Error: File '{file_path}' not found.")
        return None

    with open(file_path, 'r', encoding='utf-8') as f:
        return f.readlines()




In [36]:
#Task 1.3 — Data Validation and Filtering

In [37]:
def validate_and_filter(transactions, region=None, min_amount=None, max_amount=None):
    """
    Validates transactions and applies optional filters.
    """
    valid_transactions = []
    invalid_count = 0

    # Summary counters
    filter_summary = {
        "total_input": len(transactions),
        "invalid": 0,
        "filtered_by_region": 0,
        "filtered_by_amount": 0,
        "final_count": 0
    }

    # Display available regions
    available_regions = sorted(set(t.get("Region") for t in transactions if "Region" in t))
    print("Available regions:", available_regions)

    # Display transaction amount range
    amounts = [
        t["Quantity"] * t["UnitPrice"]
        for t in transactions
        if isinstance(t.get("Quantity"), int) and isinstance(t.get("UnitPrice"), float)
    ]

    if amounts:
        print(f"Transaction amount range: Min = {min(amounts)}, Max = {max(amounts)}")

    for t in transactions:
        # ---- Validation Rules ----
        if (
            t.get("Quantity") is None or t.get("Quantity") <= 0 or
            t.get("UnitPrice") is None or t.get("UnitPrice") <= 0 or
            not t.get("TransactionID", "").startswith("T") or
            not t.get("ProductID", "").startswith("P") or
            not t.get("CustomerID", "").startswith("C") or
            not t.get("Region")
        ):
            invalid_count += 1
            continue

        transaction_amount = t["Quantity"] * t["UnitPrice"]

        # ---- Region Filter ----
        if region and t["Region"] != region:
            filter_summary["filtered_by_region"] += 1
            continue

        # ---- Amount Filters ----
        if min_amount is not None and transaction_amount < min_amount:
            filter_summary["filtered_by_amount"] += 1
            continue

        if max_amount is not None and transaction_amount > max_amount:
            filter_summary["filtered_by_amount"] += 1
            continue

        valid_transactions.append(t)

    # Final counts
    filter_summary["invalid"] = invalid_count
    filter_summary["final_count"] = len(valid_transactions)

    return valid_transactions, invalid_count, filter_summary


In [38]:
valid_txns, invalid_count, summary = validate_and_filter(
    parsed_transactions,
    region="North",
    min_amount=1000,
    max_amount=100000
)


Available regions: ['', 'East', 'North', 'South', 'West']
Transaction amount range: Min = -8982.0, Max = 818960.0


In [39]:
print("Invalid transactions:", invalid_count)
print("Summary:", summary)
print("First valid transaction:", valid_txns[:1])


Invalid transactions: 10
Summary: {'total_input': 80, 'invalid': 10, 'filtered_by_region': 49, 'filtered_by_amount': 5, 'final_count': 16}
First valid transaction: [{'TransactionID': 'T023', 'Date': '2024-12-09', 'ProductID': 'P109', 'ProductName': 'Wireless Mouse', 'Quantity': 9, 'UnitPrice': 523.0, 'CustomerID': 'C022', 'Region': 'North'}]
